In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import statsmodels.api as sm


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Import and append the data

csv_path = '/kaggle/input/canandian-labour-force-survey/*.csv'

# Get a list of all CSV files in the specified path
file_list = glob.glob(csv_path)

# Initialize an empty list to store the dataframes
dataframes = []

# Read each CSV file and append its contents to the list
for file in file_list:
    df = pd.read_csv(file)
    dataframes.append(df)

# Combine all dataframes into a single dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

#Remove "Not in labour force" from dataset
combined_df = combined_df[combined_df['LFSSTAT'] != 4]


In [ ]:
##Replace categorical values with labels

# Define the mapping of categorical values to their corresponding text replacements
CMA_mapping = {
1:'Québec',
2:'Montréal',
3:'Ottawa–Gatineau (Ontario part)',
4:'Toronto',
5:'Hamilton',
6:'Winnipeg',
7:'Calgary',
8:'Edmonton',
9:'Vancouver',
0:'Other CMA or non-CMA'
}

AGE_12_mapping = {
1:'Gen Z',
2:'Gen Z',
3:'Millennials',
4:'Millennials',
5:'Millennials',
6:'Gen X',
7:'Gen X',
8:'Gen X',
9:'Baby Boomer',
10:'Baby Boomer',
11:'Baby Boomer',
12:'Baby Boomer or older'
}

SEX_mapping={
1:'Male',
2:'Female'}

EDUC_mapping={
0:'0 to 8 years',
1:'Some high school',
2:'High school graduate',
3:'Some postsecondary',
4:'Postsecondary certificate or diploma',
5:'Bachelors degree',
6:'Above bachelors degree'
}

IMMIG_mapping={
1:'Immigrant, landed 10 or less years earlier',
2:'Immigrant, landed more than 10 years earlier',
3:'Non-immigrant'}

EFAMTYPE_mapping={
1:'Person not in an economic family',
2:'Dual-earner couple, no children or none under 25',
3:'Dual-earner couple, youngest child 0 to 17',
4:'Dual-earner couple, youngest child 18 to 24',
5:'Single-earner couple, male employed, no children or none under 25',
6:'Single-earner couple, male employed, youngest child 0 to 17',
7:'Single-earner couple, male employed, youngest child 18 to 24',
8:'Single-earner couple, female employed, no children or none under 25',
9:'Single-earner couple, female employed, youngest child 0 to 17',
10:'Single-earner couple, female employed, youngest child 18 to 24',
11:'Non-earner couple, no children or none under 25',
12:'Non-earner couple, youngest child 0 to 17',
13:'Non-earner couple, youngest child 18 to 24',
14:'Lone-parent family, parent employed, youngest child 0 to 17',
15:'Lone-parent family, parent employed, youngest child 18 to 24',
16:'Lone-parent family, parent not employed, youngest child 0 to 17',
17:'Lone-parent family, parent not employed, youngest child 18 to 24',
18:'Other families'}

NAICS_21_mapping = {
1:'Agriculture',
2:'Forestry and logging and support activities for forestry',
3:'Fishing, hunting and trapping',
4:'Mining, quarrying, and oil and gas extraction',
5:'Utilities',
6:'Construction',
7:'Manufacturing - durable goods',
8:'Manufacturing - non-durable goods',
9:'Wholesale trade',
10:'Retail trade',
11:'Transportation and warehousing',
12:'Finance and insurance',
13:'Real estate and rental and leasing',
14:'Professional, scientific and technical services',
15:'Business, building and other support services',
16:'Educational services',
17:'Health care and social assistance',
18:'Information, culture and recreation',
19:'Accommodation and food services',
20:'Other services (except public administration)',
21:'Public administration'
}

PERMTEMP_mapping = {
1:'Permanent',
2:'Temporary, seasonal job',
3:'Temporary, term or contract job',
4:'Temporary, casual or other temporary jobs'
}

FLOWUNEM_mapping={
1:'Job losers, temporary layoff',
2:'Job losers, permanent layoff',
3:'Job leavers',
4:'Job leavers/losers (status unknown), worked more than 1 year ago ',
5:'New entrants',
6:'Re-entrants, worked 1 year ago or less',
7:'Re-entrants, worked more than 1 year ago',
8:'Future starts'
}

WHYLEFTO_mapping={
0:'Job leavers, other reasons',
1:'Job leavers, own illness or disability',
2:'Job leavers, personal or family responsibilities',
3:'Job leavers, going to school',
4:'Job losers, laid off',
5:'Job leavers, retired'
}

NOC_43_mapping = {
1:'Legislative and senior management occupations',
2:'Specialized middle management occupations',
3:'Middle management occupations in retail and wholesale trade and customer services',
4:'Middle management occupations in trades, transportation, production and utilities',
5:'Professional occupations in finance',
6:'Professional occupations in business',
7:'Administrative and financial supervisors and specialized administrative occupations',
8:'Administrative occupations and transportation logistics occupations',
9:'Administrative and financial support and supply chain logistics occupations',
10:'Professional occupations in natural sciences',
11:'Professional occupations in applied sciences (except engineering)',
12:'Professional occupations in engineering',
13:'Technical occupations related to natural and applied sciences',
14:'Health treating and consultation services professionals',
15:'Therapy and assessment professionals',
16:'Nursing and allied health professionals',
17:'Technical occupations in health',
18:'Assisting occupations in support of health services',
19:'Professional occupations in law',
20:'Professional occupations in education services',
21:'Professional occupations in social and community services',
22:'Professional occupations in government services',
23:'Occupations in front-line public protection services',
24:'Paraprofessional occupations in legal, social, community and education services',
25:'Assisting occupations in education and in legal and public protection',
26:'Care providers and public protection support occupations and student monitors, crossing guards and related occupations',
27:'Professional occupations in art and culture',
28:'Technical occupations in art, culture and sport',
29:'Occupations in art, culture and sport',
30:'Support occupations in art, culture and sport',
31:'Retail sales and service supervisors and specialized occupations in sales and services',
32:'Occupations in sales and services',
33:'Sales and service representatives and other customer and personal services occupations',
34:'Sales and service support occupations',
35:'Technical trades and transportation officers and controllers',
36:'General trades',
37:'Mail and message distribution, other transport equipment operators and related maintenance workers',
38:'Helpers and labourers and other transport drivers, operators and labourers',
39:'Supervisors and occupations in natural resources, agriculture and related production',
40:'Workers and labourers in natural resources, agriculture and related production',
41:'Supervisors, central control and process operators in processing, manufacturing and utilities and aircraft assemblers and inspectors',
42:'Machine operators, assemblers and inspectors in processing, manufacturing and printing',
43:'Labourers in processing, manufacturing and utilities'}

# Replace the categorical values with their corresponding text replacements
combined_df['CMA'] = combined_df['CMA'].replace(CMA_mapping)
combined_df['AGE_12'] = combined_df['AGE_12'].replace(AGE_12_mapping)
combined_df['SEX'] = combined_df['SEX'].replace(SEX_mapping)
combined_df['EDUC'] = combined_df['EDUC'].replace(EDUC_mapping)
combined_df['IMMIG'] = combined_df['IMMIG'].replace(IMMIG_mapping)
combined_df['EFAMTYPE'] = combined_df['EFAMTYPE'].replace(EFAMTYPE_mapping)
combined_df['NAICS_21'] = combined_df['NAICS_21'].replace(NAICS_21_mapping)
combined_df['PERMTEMP'] = combined_df['PERMTEMP'].replace(PERMTEMP_mapping).fillna('N/A')
combined_df['FLOWUNEM'] = combined_df['FLOWUNEM'].replace(FLOWUNEM_mapping)
combined_df['WHYLEFTO'] = combined_df['WHYLEFTO'].replace(WHYLEFTO_mapping)
combined_df['NOC_43'] = combined_df['NOC_43'].replace(NOC_43_mapping)


# Annualize the earnings
combined_df['HRLYEARN']=combined_df['HRLYEARN'].div(100).mul(52).mul(40).astype(float).round(2)

print(combined_df.head(5))


# In this analysis, we will answer the following questions about:

Scope of this analysis

# How has the demographics of the Canadian labour market looks over the past 5 years?
In this part, we will look at the labour market from the sex lens, the immigration lens,and the age lens.

In [ ]:
# Canada, Gender distribution

# Calculate the total count per column
column_totals = combined_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table_casex = combined_df.pivot_table(index='SEX', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table_casex = (pivot_table_casex.div(column_totals, axis=1) * 100).astype(float).round(1)

# Print the pivot table
print(pivot_table_casex)

# Create a line chart 
transposed_pivot = pivot_table_casex.T
ax = transposed_pivot.plot(kind='line', marker='o', legend=False)
plt.gca().xaxis.set_major_formatter('{:.0f}'.format)

# Label each line with IMMIG values
for age in pivot_table_casex.index:
    line_data = transposed_pivot[age]
    last_value = line_data.dropna().iloc[-1]
    ax.text(line_data.index[-1], last_value, age, ha='right', va='bottom')


# Customize the plot properties
ax.set_xlabel('Year')
ax.set_ylabel('% of labour force')
ax.set_title('Canada Labour Force Sex Distribution')

# Display the plot
plt.show()

In [ ]:
# Canada, Immigrant background

# Calculate the total count per column
column_totals = combined_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table_caimmig = combined_df.pivot_table(index='IMMIG', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table_caimmig = (pivot_table_caimmig.div(column_totals, axis=1) * 100).astype(float).round(1)

# Print the pivot table
print(pivot_table_caimmig)

# Create a line chart 
transposed_pivot = pivot_table_caimmig.T
ax = transposed_pivot.plot(kind='line', marker='o', legend=False)
plt.gca().xaxis.set_major_formatter('{:.0f}'.format)

# Label each line with IMMIG values
for age in pivot_table_caimmig.index:
    line_data = transposed_pivot[age]
    last_value = line_data.dropna().iloc[-1]
    ax.text(line_data.index[-1], last_value, age, ha='right', va='bottom')


# Customize the plot properties
ax.set_xlabel('Year')
ax.set_ylabel('% of labour force')
ax.set_title('Canada Labour Force Immigrant Background Distribution')

# Display the plot
plt.show()

In [ ]:
# Canada, Age distribution

# Calculate the total count per column
column_totals = combined_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table_caage = combined_df.pivot_table(index='AGE_12', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table_caage = (pivot_table_caage.div(column_totals, axis=1) * 100).astype(float).round(1)

# Define the desired order of the index values
age_desired_order = ['Gen Z','Millennials','Gen X','Baby Boomer','Baby Boomer or older']

# Reindex the pivot table based on the desired order of index values
pivot_table_caage = pivot_table_caage.reindex(age_desired_order)

# Print the pivot table
print(pivot_table_caage)

# Create a line chart 
transposed_pivot = pivot_table_caage.T
ax = transposed_pivot.plot(kind='line', marker='o', legend=False)
plt.gca().xaxis.set_major_formatter('{:.0f}'.format)

# Label each line with AGE_12 values
for age in pivot_table_caage.index:
    line_data = transposed_pivot[age]
    last_value = line_data.dropna().iloc[-1]
    ax.text(line_data.index[-1], last_value, age, ha='right', va='bottom')

# Customize the plot properties
ax.set_xlabel('Year')
ax.set_ylabel('% of labour force')
ax.set_title('Canada Labour Force Age Distribution')

# Display the plot
plt.show()

Overall speaking, the 2022 Canadian labour market consists of:
* 48.5% female
* 51.5% male
* The split is stable over the past 5 years.

And
* 5.3% Immigrants landed 10 or less years earlier
* 13.5% Immigrants landed more than 10 years earlier
* 81.2% Non-immigrant
* There are more immigrants over the past 5 years.

And
* Baby boomers are still the largest members in the labour force, but has declined from 27.4% to 25.4% since 2021
* Millennials and Gen Z are on the rise since 2021


These numbers should be used as benchmarks when we compare a certain occupation or industry against the overall labour market. We can draw summaries such as "The oil and gas industry has less female and less immigrant than the industry average."

# Demographics of the professional tech labour market in Canada over the past 5 years

In [ ]:
# Are we seeing growth in certain occupations?

# Calculate the total count per column
column_totals = combined_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table = combined_df.pivot_table(index='NOC_43', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(1)

# Sort the values in ascending order based on value
sorted_pivot_table = pivot_table.sort_values(by=2022, ascending=False)

# Print the sorted pivot table
print(sorted_pivot_table)

# Calculate the difference between all columns
pivot_table_diff = pivot_table.diff(axis=1)

# Sort the values in ascending order based on value
pivot_table_diff = pivot_table_diff.sort_values(by=2022, ascending=False)

# Create a heatmap of the pivot table
plt.figure(figsize=(10, 20))
sns.heatmap(pivot_table_diff, cmap='vlag_r', annot=True, fmt='.1f', cbar=True)

# Set the plot title and labels
plt.title('Are we seeing growth in certain occupations?')
plt.xlabel('Year')
plt.ylabel('NOC_43')

# Display the heatmap
plt.show()

## NOC_43 code indicates their occupation at main job. Here's the list:
> 1. Legislative and senior management occupations
> 1. Specialized middle management occupations
> 1. Middle management occupations in retail and wholesale trade and customer services
> 1. Middle management occupations in trades, transportation, production and utilities
> 1. Professional occupations in finance
> 1. Professional occupations in business
> 1. Administrative and financial supervisors and specialized administrative occupations
> 1. Administrative occupations and transportation logistics occupations
> 1. Administrative and financial support and supply chain logistics occupations
> 1. Professional occupations in natural sciences
> 1. Professional occupations in applied sciences (except engineering)
> 1. Professional occupations in engineering
> 1. Technical occupations related to natural and applied sciences
> 1. Health treating and consultation services professionals
> 1. Therapy and assessment professionals
> 1. Nursing and allied health professionals
> 1. Technical occupations in health
> 1. Assisting occupations in support of health services
> 1. Professional occupations in law
> 1. Professional occupations in education services
> 1. Professional occupations in social and community services
> 1. Professional occupations in government services
> 1. Occupations in front-line public protection services
> 1. Paraprofessional occupations in legal, social, community and education services
> 1. Assisting occupations in education and in legal and public protection
> 1. Care providers and public protection support occupations and student monitors, crossing guards and related occupations
> 1. Professional occupations in art and culture
> 1. Technical occupations in art, culture and sport
> 1. Occupations in art, culture and sport
> 1. Support occupations in art, culture and sport
> 1. Retail sales and service supervisors and specialized occupations in sales and services
> 1. Occupations in sales and services
> 1. Sales and service representatives and other customer and personal services occupations
> 1. Sales and service support occupations
> 1. Technical trades and transportation officers and controllers
> 1. General trades
> 1. Mail and message distribution, other transport equipment operators and related maintenance workers
> 1. Helpers and labourers and other transport drivers, operators and labourers
> 1. Supervisors and occupations in natural resources, agriculture and related production
> 1. Workers and labourers in natural resources, agriculture and related production
> 1. Supervisors, central control and process operators in processing, manufacturing and utilities and aircraft assemblers and inspectors
> 1. Machine operators, assemblers and inspectors in processing, manufacturing and printing
> 1. Labourers in processing, manufacturing and utilities

In [ ]:
#Filter by the National Occupation Classification
#Enter which NOC(s) you would like to see
#In my case, I want to know the labour market for 11: Professional occupations in applied sciences (except engineering)
filter_noc = ['Professional occupations in applied sciences (except engineering)']

# Filter the dataframe based on the values in the 'NOC_43' column
filtered_noc_df = combined_df[combined_df['NOC_43'].isin(filter_noc)]
# Filter by only full-time employees (1=full-time, 2=part-time)
filtered_noc_df = filtered_noc_df[filtered_noc_df['FTPTMAIN']==1]

# Print the combined dataframe
pd.set_option('display.max_columns', None)
print(filtered_noc_df.head(5))


In [ ]:
# Tech profession, noc_11, Gender distribution

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table_nocsex = filtered_noc_df.pivot_table(index='SEX', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table_nocsex = (pivot_table_nocsex.div(column_totals, axis=1) * 100).astype(float).round(0)

# Print the pivot table
print(pivot_table_nocsex)

In [ ]:
# Tech profession, noc_11, Immigrant background

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table_nocimmig = filtered_noc_df.pivot_table(index='IMMIG', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table_nocimmig = (pivot_table_nocimmig.div(column_totals, axis=1) * 100).astype(float).round(0)

# Print the pivot table
print(pivot_table_nocimmig)

In [ ]:
# Tech profession, noc_11, Educational level

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table = filtered_noc_df.pivot_table(index='EDUC', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(0)

# Print the pivot table
print(pivot_table)

In [ ]:
# Tech profession, noc_11, Age distribution

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table_nocage = filtered_noc_df.pivot_table(index='AGE_12', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table_nocage = (pivot_table_nocage.div(column_totals, axis=1) * 100).astype(float).round(0)
# Reindex the pivot table based on the desired order of index values
pivot_table_nocage = pivot_table_nocage.reindex(age_desired_order)
# Print the pivot table
print(pivot_table_nocage)

In [ ]:
# Tech profession, noc_11, Economic Family Type Distribution

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table = filtered_noc_df.pivot_table(index='EFAMTYPE', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(0)

# Sort the values in ascending order based on value
sorted_pivot_table = pivot_table.sort_values(by=2022, ascending=False)

# Print the sorted pivot table
print(sorted_pivot_table)

# How does the tech profession demographics compare to the national average?

In [ ]:
#Comparison
#If coef > 1, then more likely
#If coef < 1, then less likely

sex_coef = pivot_table_nocsex/pivot_table_casex
immig_coef = pivot_table_nocimmig/pivot_table_caimmig
age_coef = pivot_table_nocage/pivot_table_caage

print(sex_coef)
print(immig_coef)
print(age_coef)

### Your typical tech profession worker has a:
* 73% of being a male at birth
* 40% chance of being an immigrant
* 42% chance of being a Millenial
* 23% chance of being a Gen X

### and is this many times more likely than the Canadian average to be:
* 1.5 times more likely to be a male at birth
* 2.1 times more likely to be an immigrant
* 1.9 times more likely to be a Millenial
* 1.5 times more likely to be a Gen X

# Tech talent clusters: where are they?

In [ ]:
# Tech profession, noc_11, CMA residence distribution

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with percentage calculation
pivot_table = filtered_noc_df.pivot_table(index='CMA', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(1)

# Sort the values in ascending order based on value
sorted_pivot_table = pivot_table.sort_values(by=2022, ascending=False)

# Print the sorted pivot table
print(sorted_pivot_table)

# Calculate the difference between all columns
pivot_table_diff = pivot_table.diff(axis=1)

# Sort the values in ascending order based on value
pivot_table_diff = pivot_table_diff.sort_values(by=2022, ascending=False)

# Print the pivot table with the difference between columns
print(pivot_table_diff)

# Create a heatmap of the pivot table
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table_diff, cmap='vlag_r', annot=True, fmt='.1f', cbar=True)

# Set the plot title and labels
plt.title('Which CMA is becoming more/less popular for tech workers?')
plt.xlabel('Year')
plt.ylabel('CMA')

# Display the heatmap
plt.show()

### Top 3 Tech talent cluster by size in 2022 (largest to smallest):
1. Toronto
1. Vancouver
1. Montreal

### Growing tech clusters (from 2021 to 2022)
1. Toronto
1. Vancouver
1. Montreal

The big three tech clusters in Canada are growing steadily and remain the top clusters in Canada.

In [ ]:
# What industry do the tech worker works in each CMA?

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('CMA').size()

# Create the pivot table with percentage calculation
pivot_table = filtered_noc_df.pivot_table(index='NAICS_21', columns='CMA', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float)

# Create a heatmap of the pivot table
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, cmap='mako_r', annot=True, fmt='.1f', cbar=True)

# Set the plot title and labels
plt.title('What industry do the tech workers work in each CMA?')
plt.xlabel('CMA')
plt.ylabel('Industry')

# Display the heatmap
plt.show()

In [ ]:
# Average earning in tech professions - distribution 2022

# Plot distribution for full-time 2022
filtered_noc_df_2022=filtered_noc_df[filtered_noc_df['SURVYEAR']==2022]
plt.hist(filtered_noc_df_2022['HRLYEARN'], bins=100)

# Add a text annotation for the mode
mode_value = filtered_noc_df_2022['HRLYEARN'].mode().values[0]
max_frequency = plt.gca().get_ylim()[1]
plt.text(mode_value, max_frequency, f"Mode: {mode_value}", ha='left', va='top')

plt.xlabel('Annual Income')
plt.ylabel('Frequency')
plt.title('Distribution of annual income of full-time tech professionals in 2022')
plt.show()

# Calculate summary statistics 2022 salary
summary = filtered_noc_df_2022['HRLYEARN'].describe().round(2)
print(summary)

# Average earning in tech professions - by CMA

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR').size()

# Create the pivot table with average hourly wage
pivot_table = filtered_noc_df.pivot_table(index='CMA', columns='SURVYEAR', values = 'HRLYEARN', aggfunc='mean', fill_value=0).round(2)

# Sort the values in ascending order based on value
pivot_table = pivot_table.sort_values(by=2022, ascending=False)

# Print the sorted pivot table
print(f"Average annual salary for tech workers: \n{pivot_table}\n")

# Which city has the largest increase in wage from 2021 to 2022?
percentage_difference = ((pivot_table[2022] - pivot_table[2021]) / pivot_table[2021]) * 100
percentage_difference = percentage_difference.sort_values(ascending=False).round(1)

# Print the percentage difference
print(f"Percentage change of average salary from 2021 to 2022: \n{percentage_difference}\n")

# Which city has the largest increase in wage from 2022 to 2023?
percentage_difference = ((pivot_table[2023] - pivot_table[2022]) / pivot_table[2022]) * 100
percentage_difference = percentage_difference.sort_values(ascending=False).round(1)

# Print the percentage difference
print(f"Percentage change of average salary from 2022 to 2023: \n{percentage_difference}\n")

In [ ]:
# Average earning in tech professions for each educational level in each CMA

# Get unique values of the 'Category' variable
education = filtered_noc_df['EDUC'].unique()

# Generate pivot tables for each category
pivot_tables = {}
for education in education:
    pivot_table = filtered_noc_df[filtered_noc_df['EDUC'] == education].pivot_table(index='CMA', columns='SURVYEAR', values = 'HRLYEARN', aggfunc='mean', fill_value=0)
    pivot_tables[education] = pivot_table

# Output the pivot tables
for education, pivot_table in pivot_tables.items():
    print(f"Average annual salary for {education}:\n{pivot_table}\n")
    
# Which city has the largest increase in wage from 2021 to 2022?
percentage_differences = {}
percentage_difference = ((pivot_table[2022] - pivot_table[2021]) / pivot_table[2021]) * 100
    
# Output the pivot tables
for education, pivot_table in pivot_tables.items():
    print(percentage_difference)
    


### Earnings from the highest to lowest by CMA
1. Calgary
1. Toronto
1. Ottawa–Gatineau (Ontario part)

### Using other CMA or non-CMA as the "normal" raise in wage, the following cities experienced exceptional wage increase
1. Toronto (12.8%)
1. Calgary (10.2%)
1. Edmonton (9%)
1. Ottawa–Gatineau (Ontario part) (7.3%)
1. Québec (5.9%)

# Unemployment

In [ ]:
# Why did Canadians leave the job?

# Calculate the total count per column
column_totals = combined_df.groupby('SURVYEAR')['WHYLEFTO'].count()

# Create the pivot table with percentage calculation
pivot_table = combined_df.pivot_table(index='WHYLEFTO', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(1)

print(f"Why did Canadians leave their jobs? \n{pivot_table}\n")

In [ ]:
# How did they become unemployed?

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR')['FLOWUNEM'].count()

# Create the pivot table with percentage calculation
pivot_table = filtered_noc_df.pivot_table(index='FLOWUNEM', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(1)

print(f"How did the tech workers become unemployed? \n{pivot_table}\n")

In [ ]:
# Why did they leave the job?

# Calculate the total count per column
column_totals = filtered_noc_df.groupby('SURVYEAR')['WHYLEFTO'].count()

# Create the pivot table with percentage calculation
pivot_table = filtered_noc_df.pivot_table(index='WHYLEFTO', columns='SURVYEAR', aggfunc='size', fill_value=0)

# Divide each value by the total count and multiply by 100 to get the percentage
pivot_table = (pivot_table.div(column_totals, axis=1) * 100).astype(float).round(1)

print(f"Why did the tech workers leave their jobs? \n{pivot_table}\n")